# Advanced Query Types

In this notebook, we will explore advanced query types available in RedisVL:

1. **`TextQuery`**: Full text search with advanced scoring
2. **`AggregateHybridQuery`**: Combines text and vector search for hybrid retrieval
3. **`MultiVectorQuery`**: Search over multiple vector fields simultaneously

These query types are powerful tools for building sophisticated search applications that go beyond simple vector similarity search.

Prerequisites:
- Ensure RedisVL4J is available in your Java environment.
- Have a running instance of [Redis Stack](https://redis.io/docs/latest/operate/oss_and_stack/install/install-stack/) or [Redis Cloud](https://redis.io/cloud).

## Setup and Data Preparation

First, let's create a schema and prepare sample data that includes text fields, numeric fields, and vector fields.

In [ ]:
// Load Maven dependencies
%maven redis.clients:jedis:7.0.0
%maven org.slf4j:slf4j-nop:2.0.16
%maven com.fasterxml.jackson.core:jackson-databind:2.18.0
%maven com.fasterxml.jackson.dataformat:jackson-dataformat-yaml:2.18.0

// Import RedisVL classes
import com.redis.vl.index.SearchIndex;
import com.redis.vl.schema.*;
import com.redis.vl.query.*;

// Import Redis client
import redis.clients.jedis.UnifiedJedis;
import redis.clients.jedis.HostAndPort;

// Import Java standard libraries
import java.util.*;
import java.nio.*;

In [ ]:
// Helper method to convert float arrays to byte arrays for vector fields
byte[] floatArrayToBytes(float[] vector) {
    ByteBuffer buffer = ByteBuffer.allocate(vector.length * 4).order(ByteOrder.LITTLE_ENDIAN);
    for (float value : vector) {
        buffer.putFloat(value);
    }
    return buffer.array();
}

## Sample Data with Text Descriptions and Vectors

We'll create sample product data with text descriptions, categories, prices, and two types of embeddings:
- **text_embedding**: 3-dimensional vectors representing product text
- **image_embedding**: 2-dimensional vectors representing product images

In [ ]:
// Sample data with text descriptions, categories, and vectors (matching Python exactly)
List<Map<String, Object>> data = Arrays.asList(
    Map.of(
        "product_id", "prod_1",
        "brief_description", "comfortable running shoes for athletes",
        "full_description", "Engineered with a dual-layer EVA foam midsole and FlexWeave breathable mesh upper, these running shoes deliver responsive cushioning for long-distance runs. The anatomical footbed adapts to your stride while the carbon rubber outsole provides superior traction on varied terrain.",
        "category", "footwear",
        "price", "89.99",
        "rating", "4.5",
        "text_embedding", floatArrayToBytes(new float[]{0.1f, 0.2f, 0.1f}),
        "image_embedding", floatArrayToBytes(new float[]{0.8f, 0.1f})
    ),
    Map.of(
        "product_id", "prod_2",
        "brief_description", "lightweight running jacket with water resistance",
        "full_description", "Stay protected with this ultralight 2.5-layer DWR-coated shell featuring laser-cut ventilation zones and reflective piping for low-light visibility. Packs into its own chest pocket and weighs just 4.2 oz, making it ideal for unpredictable weather conditions.",
        "category", "outerwear",
        "price", "129.99",
        "rating", "4.8",
        "text_embedding", floatArrayToBytes(new float[]{0.2f, 0.3f, 0.2f}),
        "image_embedding", floatArrayToBytes(new float[]{0.7f, 0.2f})
    ),
    Map.of(
        "product_id", "prod_3",
        "brief_description", "professional tennis racket for competitive players",
        "full_description", "Competition-grade racket featuring a 98 sq in head size, 16x19 string pattern, and aerospace-grade graphite frame that delivers explosive power with pinpoint control. Tournament-approved specs include 315g weight and 68 RA stiffness rating for advanced baseline play.",
        "category", "equipment",
        "price", "199.99",
        "rating", "4.9",
        "text_embedding", floatArrayToBytes(new float[]{0.9f, 0.1f, 0.05f}),
        "image_embedding", floatArrayToBytes(new float[]{0.1f, 0.9f})
    ),
    Map.of(
        "product_id", "prod_4",
        "brief_description", "yoga mat with extra cushioning for comfort",
        "full_description", "Premium 8mm thick TPE yoga mat with dual-texture surface - smooth side for hot yoga flow and textured side for maximum grip during balancing poses. Closed-cell technology prevents moisture absorption while alignment markers guide proper positioning in asanas.",
        "category", "accessories",
        "price", "39.99",
        "rating", "4.3",
        "text_embedding", floatArrayToBytes(new float[]{0.15f, 0.25f, 0.15f}),
        "image_embedding", floatArrayToBytes(new float[]{0.5f, 0.5f})
    ),
    Map.of(
        "product_id", "prod_5",
        "brief_description", "basketball shoes with excellent ankle support",
        "full_description", "High-top basketball sneakers with Zoom Air units in forefoot and heel, reinforced lateral sidewalls for explosive cuts, and herringbone traction pattern optimized for hardwood courts. The internal bootie construction and extended ankle collar provide lockdown support during aggressive drives.",
        "category", "footwear",
        "price", "139.99",
        "rating", "4.7",
        "text_embedding", floatArrayToBytes(new float[]{0.12f, 0.18f, 0.12f}),
        "image_embedding", floatArrayToBytes(new float[]{0.75f, 0.15f})
    ),
    Map.of(
        "product_id", "prod_6",
        "brief_description", "swimming goggles with anti-fog coating",
        "full_description", "Low-profile competition goggles with curved polycarbonate lenses offering 180-degree peripheral vision and UV protection. Hydrophobic anti-fog coating lasts 10x longer than standard treatments, while the split silicone strap and interchangeable nose bridges ensure a watertight, custom fit.",
        "category", "accessories",
        "price", "24.99",
        "rating", "4.4",
        "text_embedding", floatArrayToBytes(new float[]{0.3f, 0.1f, 0.2f}),
        "image_embedding", floatArrayToBytes(new float[]{0.2f, 0.8f})
    )
);

System.out.println("Created " + data.size() + " sample products");

## Define the Schema

Our schema includes:
- **Tag fields**: `product_id`, `category`
- **Text fields**: `brief_description` and `full_description` for full-text search
- **Numeric fields**: `price`, `rating`
- **Vector fields**: `text_embedding` (3 dimensions) and `image_embedding` (2 dimensions) for semantic search

In [ ]:
IndexSchema schema = IndexSchema.builder()
    .name("advanced_queries")
    .prefix("products")
    .storageType(IndexSchema.StorageType.HASH)
    .field(TagField.builder().name("product_id").build())
    .field(TagField.builder().name("category").build())
    .field(TextField.builder().name("brief_description").build())
    .field(TextField.builder().name("full_description").build())
    .field(NumericField.builder().name("price").build())
    .field(NumericField.builder().name("rating").build())
    .field(
        VectorField.builder()
            .name("text_embedding")
            .dimensions(3)
            .distanceMetric(VectorField.DistanceMetric.COSINE)
            .build())
    .field(
        VectorField.builder()
            .name("image_embedding")
            .dimensions(2)
            .distanceMetric(VectorField.DistanceMetric.COSINE)
            .build())
    .build();

System.out.println("Schema created");

## Create Index and Load Data

In [ ]:
// Connect to Redis
UnifiedJedis client = new UnifiedJedis(new HostAndPort("localhost", 6379));

// Create the search index
SearchIndex index = new SearchIndex(schema, client);
index.create(true);

// Load data
List<String> keys = new ArrayList<>();
for (Map<String, Object> product : data) {
    String key = "products:" + product.get("product_id");
    keys.add(key);
    Map<String, String> fields = new HashMap<>();
    product.forEach((k, v) -> {
        if (v instanceof byte[]) {
            fields.put(k, new String((byte[]) v, java.nio.charset.StandardCharsets.ISO_8859_1));
        } else {
            fields.put(k, String.valueOf(v));
        }
    });
    client.hset(key, fields);
}

System.out.println("Loaded " + keys.size() + " products into the index");

## 1. TextQuery: Full Text Search

The `TextQuery` class enables full text search with advanced scoring algorithms. It's ideal for keyword-based search with relevance ranking.

### Basic Text Search

Let's search for products related to "running shoes":

In [ ]:
TextQuery textQuery = TextQuery.builder()
    .text("running shoes")
    .textField("brief_description")
    .returnFields(Arrays.asList("product_id", "brief_description", "category", "price"))
    .numResults(5)
    .build();

List<Map<String, Object>> results = index.query(textQuery);

System.out.println("Found " + results.size() + " results:");
for (Map<String, Object> result : results) {
    System.out.println("  " + result.get("product_id") + ": " + result.get("brief_description"));
}

### Text Search with Different Scoring Algorithms

RedisVL supports multiple text scoring algorithms. Let's compare `BM25STD` and `TFIDF`:

In [ ]:
// BM25 standard scoring (default)
TextQuery bm25Query = TextQuery.builder()
    .text("comfortable shoes")
    .textField("brief_description")
    .scorer("BM25STD")
    .returnFields(Arrays.asList("product_id", "brief_description", "price"))
    .numResults(3)
    .build();

System.out.println("Results with BM25 scoring:");
List<Map<String, Object>> bm25Results = index.query(bm25Query);
for (Map<String, Object> result : bm25Results) {
    System.out.println("  " + result.get("product_id") + ": " + result.get("brief_description"));
}

In [ ]:
// TFIDF scoring
TextQuery tfidfQuery = TextQuery.builder()
    .text("comfortable shoes")
    .textField("brief_description")
    .scorer("TFIDF")
    .returnFields(Arrays.asList("product_id", "brief_description", "price"))
    .numResults(3)
    .build();

System.out.println("Results with TFIDF scoring:");
List<Map<String, Object>> tfidfResults = index.query(tfidfQuery);
for (Map<String, Object> result : tfidfResults) {
    System.out.println("  " + result.get("product_id") + ": " + result.get("brief_description"));
}

### Text Search with Filters

Combine text search with filters to narrow results:

In [ ]:
// Search for "shoes" only in the footwear category
TextQuery filteredTextQuery = TextQuery.builder()
    .text("shoes")
    .textField("brief_description")
    .filterExpression(Filter.tag("category", "footwear"))
    .returnFields(Arrays.asList("product_id", "brief_description", "category", "price"))
    .numResults(5)
    .build();

List<Map<String, Object>> filteredResults = index.query(filteredTextQuery);
System.out.println("Filtered results (footwear category):");
for (Map<String, Object> result : filteredResults) {
    System.out.println("  " + result.get("brief_description"));
}

In [ ]:
// Search for products under $100
TextQuery priceFilteredQuery = TextQuery.builder()
    .text("comfortable")
    .textField("brief_description")
    .filterExpression(Filter.numeric("price").lt(100))
    .returnFields(Arrays.asList("product_id", "brief_description", "price"))
    .numResults(5)
    .build();

List<Map<String, Object>> priceResults = index.query(priceFilteredQuery);
System.out.println("Price filtered results (under $100):");
for (Map<String, Object> result : priceResults) {
    System.out.println("  " + result.get("brief_description") + " - $" + result.get("price"));
}

### Text Search with Multiple Fields and Weights

You can search across multiple text fields with different weights to prioritize certain fields.
Here we'll prioritize the `brief_description` field and make text similarity in that field twice as important as text similarity in `full_description`:

In [ ]:
Map<String, Double> fieldWeights = new HashMap<>();
fieldWeights.put("brief_description", 1.0);
fieldWeights.put("full_description", 0.5);

TextQuery weightedQuery = TextQuery.builder()
    .text("shoes")
    .textFieldWeights(fieldWeights)
    .returnFields(Arrays.asList("product_id", "brief_description"))
    .numResults(3)
    .build();

List<Map<String, Object>> weightedResults = index.query(weightedQuery);
System.out.println("Weighted field search results:");
for (Map<String, Object> result : weightedResults) {
    System.out.println("  " + result.get("brief_description"));
}

## 2. AggregateHybridQuery: Combining Text and Vector Search

The `AggregateHybridQuery` combines text search and vector similarity to provide the best of both worlds:
- **Text search**: Finds exact keyword matches
- **Vector search**: Captures semantic similarity

Results are scored using a weighted combination:

```
hybrid_score = (alpha) * vector_score + (1 - alpha) * text_score
```

Where `alpha` controls the balance between vector and text search (default: 0.7).

### Basic Aggregate Hybrid Query

Let's search for "running shoes" with both text and semantic search:

In [ ]:
AggregateHybridQuery hybridQuery = AggregateHybridQuery.builder()
    .text("running shoes")
    .textFieldName("brief_description")
    .vector(new float[]{0.1f, 0.2f, 0.1f})
    .vectorFieldName("text_embedding")
    .returnFields(Arrays.asList("product_id", "brief_description", "category", "price"))
    .numResults(5)
    .build();

List<Map<String, Object>> hybridResults = index.query(hybridQuery);

System.out.println("Hybrid search results:");
for (Map<String, Object> result : hybridResults) {
    System.out.println("  " + result.get("product_id") + ": " + result.get("brief_description"));
}

### Adjusting the Alpha Parameter

The `alpha` parameter controls the weight between vector and text search:
- `alpha=1.0`: Pure vector search
- `alpha=0.0`: Pure text search
- `alpha=0.7` (default): 70% vector, 30% text

In [ ]:
// More emphasis on vector search (alpha=0.9)
AggregateHybridQuery vectorHeavyQuery = AggregateHybridQuery.builder()
    .text("comfortable")
    .textFieldName("brief_description")
    .vector(new float[]{0.15f, 0.25f, 0.15f})
    .vectorFieldName("text_embedding")
    .alpha(0.9f)  // 90% vector, 10% text
    .returnFields(Arrays.asList("product_id", "brief_description"))
    .numResults(3)
    .build();

System.out.println("Results with alpha=0.9 (vector-heavy):");
List<Map<String, Object>> vectorHeavyResults = index.query(vectorHeavyQuery);
for (Map<String, Object> result : vectorHeavyResults) {
    System.out.println("  " + result.get("brief_description"));
}

### Aggregate Hybrid Query with Filters

You can also combine hybrid search with filters:

In [ ]:
// Hybrid search with a price filter
AggregateHybridQuery filteredHybridQuery = AggregateHybridQuery.builder()
    .text("professional equipment")
    .textFieldName("brief_description")
    .vector(new float[]{0.9f, 0.1f, 0.05f})
    .vectorFieldName("text_embedding")
    .filterExpression(Filter.numeric("price").gt(100))
    .returnFields(Arrays.asList("product_id", "brief_description", "category", "price"))
    .numResults(5)
    .build();

List<Map<String, Object>> filteredHybridResults = index.query(filteredHybridQuery);
System.out.println("Filtered hybrid results (price > $100):");
for (Map<String, Object> result : filteredHybridResults) {
    System.out.println("  " + result.get("brief_description") + " - $" + result.get("price"));
}

### Using Different Text Scorers

AggregateHybridQuery supports the same text scoring algorithms as TextQuery:

In [ ]:
// Aggregate Hybrid query with TFIDF scorer
AggregateHybridQuery hybridTfidf = AggregateHybridQuery.builder()
    .text("shoes support")
    .textFieldName("brief_description")
    .vector(new float[]{0.12f, 0.18f, 0.12f})
    .vectorFieldName("text_embedding")
    .textScorer("TFIDF")
    .returnFields(Arrays.asList("product_id", "brief_description"))
    .numResults(3)
    .build();

List<Map<String, Object>> hybridTfidfResults = index.query(hybridTfidf);
System.out.println("Hybrid query with TFIDF scorer:");
for (Map<String, Object> result : hybridTfidfResults) {
    System.out.println("  " + result.get("brief_description"));
}

## 3. MultiVectorQuery: Multi-Vector Search

The `MultiVectorQuery` allows you to search over multiple vector fields simultaneously. This is useful when you have different types of embeddings (e.g., text and image embeddings) and want to find results that match across multiple modalities.

The final score is calculated as a weighted combination:

```
combined_score = w_1 * score_1 + w_2 * score_2 + w_3 * score_3 + ...
```

### Basic Multi-Vector Query

First, we need to import the `Vector` class to define our query vectors:

In [ ]:
// Define multiple vectors for the query
com.redis.vl.query.Vector textVector = com.redis.vl.query.Vector.builder()
    .vector(new float[]{0.1f, 0.2f, 0.1f})
    .fieldName("text_embedding")
    .dtype("float32")
    .weight(0.7)  // 70% weight for text embedding
    .build();

com.redis.vl.query.Vector imageVector = com.redis.vl.query.Vector.builder()
    .vector(new float[]{0.8f, 0.1f})
    .fieldName("image_embedding")
    .dtype("float32")
    .weight(0.3)  // 30% weight for image embedding
    .build();

// Create a multi-vector query
MultiVectorQuery multiVectorQuery = MultiVectorQuery.builder()
    .vectors(textVector, imageVector)
    .returnFields(Arrays.asList("product_id", "brief_description", "category"))
    .numResults(5)
    .build();

List<Map<String, Object>> multiResults = index.query(multiVectorQuery);
System.out.println("Multi-vector search results:");
for (Map<String, Object> result : multiResults) {
    System.out.println("  " + result.get("product_id") + ": " + result.get("brief_description"));
}

### Adjusting Vector Weights

You can adjust the weights to prioritize different vector fields:

In [ ]:
// More emphasis on image similarity
com.redis.vl.query.Vector textVec = com.redis.vl.query.Vector.builder()
    .vector(new float[]{0.9f, 0.1f, 0.05f})
    .fieldName("text_embedding")
    .dtype("float32")
    .weight(0.2)  // 20% weight
    .build();

com.redis.vl.query.Vector imageVec = com.redis.vl.query.Vector.builder()
    .vector(new float[]{0.1f, 0.9f})
    .fieldName("image_embedding")
    .dtype("float32")
    .weight(0.8)  // 80% weight
    .build();

MultiVectorQuery imageHeavyQuery = MultiVectorQuery.builder()
    .vectors(textVec, imageVec)
    .returnFields(Arrays.asList("product_id", "brief_description", "category"))
    .numResults(3)
    .build();

System.out.println("Results with emphasis on image similarity:");
List<Map<String, Object>> imageHeavyResults = index.query(imageHeavyQuery);
for (Map<String, Object> result : imageHeavyResults) {
    System.out.println("  " + result.get("brief_description"));
}

### Multi-Vector Query with Filters

Combine multi-vector search with filters to narrow results:

In [ ]:
// Multi-vector search with category filter
com.redis.vl.query.Vector textVecFilter = com.redis.vl.query.Vector.builder()
    .vector(new float[]{0.1f, 0.2f, 0.1f})
    .fieldName("text_embedding")
    .dtype("float32")
    .weight(0.6)
    .build();

com.redis.vl.query.Vector imageVecFilter = com.redis.vl.query.Vector.builder()
    .vector(new float[]{0.8f, 0.1f})
    .fieldName("image_embedding")
    .dtype("float32")
    .weight(0.4)
    .build();

MultiVectorQuery filteredMultiQuery = MultiVectorQuery.builder()
    .vectors(textVecFilter, imageVecFilter)
    .filterExpression(Filter.tag("category", "footwear"))
    .returnFields(Arrays.asList("product_id", "brief_description", "category", "price"))
    .numResults(5)
    .build();

List<Map<String, Object>> filteredMultiResults = index.query(filteredMultiQuery);
System.out.println("Filtered multi-vector results (footwear only):");
for (Map<String, Object> result : filteredMultiResults) {
    System.out.println("  " + result.get("brief_description") + " (" + result.get("category") + ")");
}

## Comparing Query Types

Let's compare the two query types side by side:

In [ ]:
// TextQuery - keyword-based search
TextQuery textQ = TextQuery.builder()
    .text("shoes")
    .textField("brief_description")
    .returnFields(Arrays.asList("product_id", "brief_description"))
    .numResults(3)
    .build();

System.out.println("TextQuery Results (keyword-based):");
List<Map<String, Object>> textResults = index.query(textQ);
for (Map<String, Object> result : textResults) {
    System.out.println("  " + result.get("brief_description"));
}
System.out.println();

// MultiVectorQuery - searches multiple vector fields
com.redis.vl.query.Vector mvText = com.redis.vl.query.Vector.builder()
    .vector(new float[]{0.1f, 0.2f, 0.1f})
    .fieldName("text_embedding")
    .dtype("float32")
    .weight(0.5)
    .build();

com.redis.vl.query.Vector mvImage = com.redis.vl.query.Vector.builder()
    .vector(new float[]{0.8f, 0.1f})
    .fieldName("image_embedding")
    .dtype("float32")
    .weight(0.5)
    .build();

MultiVectorQuery multiQ = MultiVectorQuery.builder()
    .vectors(mvText, mvImage)
    .returnFields(Arrays.asList("product_id", "brief_description"))
    .numResults(3)
    .build();

System.out.println("MultiVectorQuery Results (multiple vectors):");
List<Map<String, Object>> mvResults = index.query(multiQ);
for (Map<String, Object> result : mvResults) {
    System.out.println("  " + result.get("brief_description"));
}

## Best Practices

### When to Use Each Query Type:

1. **`TextQuery`**:
   - When you need precise keyword matching
   - For traditional search engine functionality
   - When text relevance scoring is important
   - Example: Product search, document retrieval

2. **`AggregateHybridQuery`**:
   - When you want to combine keyword and semantic search
   - For improved search quality over pure text or vector search
   - When you have both text and vector representations of your data
   - Example: E-commerce search, content recommendation

3. **`MultiVectorQuery`**:
   - When you have multiple types of embeddings (text, image, audio, etc.)
   - For multi-modal search applications
   - When you want to balance multiple semantic signals
   - Example: Image-text search, cross-modal retrieval

## Cleanup

In [ ]:
// Delete the index and all data
index.delete(true);

// Close the Redis connection
client.close();

System.out.println("Cleanup complete");